In [1]:
from pathlib import Path
from pathlib import *
import pandas as pd
import re
from random import randint
from random import shuffle
import spacy
from time import time
import matplotlib.pyplot as plt
import gensim
import gensim.corpora as corpora
from pprint import pprint
from gensim.models import TfidfModel
from gensim.models import CoherenceModel
from gensim.test.utils import datapath
from gensim import  models
from gensim.models import LdaSeqModel
nlpd = spacy.load("da_core_news_lg")
import os

## Dynamic Topic-model

In [2]:
path = "../../Folketingstidende/Hele/"
dir_path = Path(f"{path}")
csv_files = list(dir_path.glob("*.csv"))

In [3]:
csv_files.sort()
csv_files[0:5]

[PosixPath('../../Folketingstidende/Hele/19531.csv'),
 PosixPath('../../Folketingstidende/Hele/19541.csv'),
 PosixPath('../../Folketingstidende/Hele/19551.csv'),
 PosixPath('../../Folketingstidende/Hele/19561.csv'),
 PosixPath('../../Folketingstidende/Hele/19562.csv')]

Jeg finder nu de forskellige filer opdelt per årti:

In [4]:
csv_files[35:49]

[PosixPath('../../Folketingstidende/Hele/19801.csv'),
 PosixPath('../../Folketingstidende/Hele/19811.csv'),
 PosixPath('../../Folketingstidende/Hele/19812.csv'),
 PosixPath('../../Folketingstidende/Hele/19821.csv'),
 PosixPath('../../Folketingstidende/Hele/19831.csv'),
 PosixPath('../../Folketingstidende/Hele/19832.csv'),
 PosixPath('../../Folketingstidende/Hele/19841.csv'),
 PosixPath('../../Folketingstidende/Hele/19851.csv'),
 PosixPath('../../Folketingstidende/Hele/19861.csv'),
 PosixPath('../../Folketingstidende/Hele/19862.csv'),
 PosixPath('../../Folketingstidende/Hele/19871.csv'),
 PosixPath('../../Folketingstidende/Hele/19872.csv'),
 PosixPath('../../Folketingstidende/Hele/19881.csv'),
 PosixPath('../../Folketingstidende/Hele/19891.csv')]

In [5]:
csv_files[49:61]

[PosixPath('../../Folketingstidende/Hele/19901.csv'),
 PosixPath('../../Folketingstidende/Hele/19902.csv'),
 PosixPath('../../Folketingstidende/Hele/19911.csv'),
 PosixPath('../../Folketingstidende/Hele/19921.csv'),
 PosixPath('../../Folketingstidende/Hele/19931.csv'),
 PosixPath('../../Folketingstidende/Hele/19941.csv'),
 PosixPath('../../Folketingstidende/Hele/19951.csv'),
 PosixPath('../../Folketingstidende/Hele/19961.csv'),
 PosixPath('../../Folketingstidende/Hele/19971.csv'),
 PosixPath('../../Folketingstidende/Hele/19972.csv'),
 PosixPath('../../Folketingstidende/Hele/19981.csv'),
 PosixPath('../../Folketingstidende/Hele/19991.csv')]

In [6]:
csv_files[61:]

[PosixPath('../../Folketingstidende/Hele/20001.csv'),
 PosixPath('../../Folketingstidende/Hele/20011.csv'),
 PosixPath('../../Folketingstidende/Hele/20012.csv'),
 PosixPath('../../Folketingstidende/Hele/20021.csv'),
 PosixPath('../../Folketingstidende/Hele/20031.csv'),
 PosixPath('../../Folketingstidende/Hele/20041.csv'),
 PosixPath('../../Folketingstidende/Hele/20042.csv'),
 PosixPath('../../Folketingstidende/Hele/20051.csv'),
 PosixPath('../../Folketingstidende/Hele/20061.csv'),
 PosixPath('../../Folketingstidende/Hele/20071.csv'),
 PosixPath('../../Folketingstidende/Hele/20072.csv'),
 PosixPath('../../Folketingstidende/Hele/20081.csv')]

In [7]:
Folketingstaler_i_80erne = {}
for fil in csv_files[35:49]:
    if str(fil)[-9:-5] in Folketingstaler_i_80erne:
        folketingsår = pd.read_csv(fil ,sep=';', encoding = "latin1")
        taler = folketingsår["indhold"]
        for tale in taler:
            Folketingstaler_i_80erne[str(fil)[-9:-5]].append(tale)
    else:
        folketingsår = pd.read_csv(fil ,sep=';', encoding = "latin1")
        taler = folketingsår["indhold"]
        Folketingstaler_i_80erne[str(fil)[-9:-5]] = []
        for tale in taler:
            Folketingstaler_i_80erne[str(fil)[-9:-5]].append(tale)

In [8]:
Folketingstaler_i_90erne = {}
for fil in csv_files[49:61]:
    if str(fil)[-9:-5] in Folketingstaler_i_90erne:
        folketingsår = pd.read_csv(fil ,sep=';', encoding = "latin1")
        taler = folketingsår["indhold"]
        for tale in taler:
            Folketingstaler_i_90erne[str(fil)[-9:-5]].append(tale)
    else:
        folketingsår = pd.read_csv(fil ,sep=';', encoding = "latin1")
        taler = folketingsår["indhold"]
        Folketingstaler_i_90erne[str(fil)[-9:-5]] = []
        for tale in taler:
            Folketingstaler_i_90erne[str(fil)[-9:-5]].append(tale)

In [9]:
Folketingstaler_i_00erne = {}
for fil in csv_files[61:]:
    if str(fil)[-9:-5] in Folketingstaler_i_00erne:
        folketingsår = pd.read_csv(fil ,sep=';', encoding = "latin1")
        taler = folketingsår["indhold"]
        for tale in taler:
            Folketingstaler_i_00erne[str(fil)[-9:-5]].append(tale)
    else:
        folketingsår = pd.read_csv(fil ,sep=';', encoding = "latin1")
        taler = folketingsår["indhold"]
        Folketingstaler_i_00erne[str(fil)[-9:-5]] = []
        for tale in taler:
            Folketingstaler_i_00erne[str(fil)[-9:-5]].append(tale)

In [10]:
def FT_taler(indhold):
    talere = []
    tale = []
    parti = []
    Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)",
                             "(KrF)", "(DR)", "(UP)", "(Grønl.)", "(Fær.)", "(DF)", "(EL)"]
    for index in range(len(indhold)):
        sentence = indhold[index]
        if len(re.findall("(^.{1,50} \([A-Z]+\)[ ]*:|^.{1,50} \(KrF\)[ ]*:|^.{1,50} \(Grønl.\)[ ]*:|^.{1,50} \(Fær.\)[ ]*:)", sentence)) > 0: #Tjekker om det er et menigt medlem
            if len(re.findall("(\([A-Z]+\)|\(KrF\)|\(Grønl.\)|\(Fær.\))", sentence)) < 2:    
                if len(re.findall("Ny stedf.", sentence)) > 0:
                    pass
                elif len(re.findall("Udtræder:", sentence)) > 0:
                    pass
                elif len(re.findall("Nyt medl", sentence)) > 0:
                    pass
                elif len(re.findall(" af", sentence.lower())) > 0:
                    pass
                elif re.findall("(\([A-Z]+\)|\(KrF\)|\(Grønl.\)|\(Fær.\))", sentence)[0] in Partier_i_folketinget:
                    if len(sentence) <= 35:
                        talere.append(sentence)
                        parti.append(re.findall("(\([A-Z]+\)|\(KrF\)|\(Grønl.\)|\(Fær.\))", sentence)[0])
                        tale.append("")
                    else: #Tager højde for læsefejl!
                        lang_taler = re.split("(^.{1,50} \([A-Z]+\)|^.{1,50} \(KrF\)|^.{1,50} \(Grønl.\)|^.{1,50} \(Fær.\))", sentence)
                        talere.append(lang_taler[-2])
                        parti.append(re.findall("(\([A-Z]+\)|\(KrF\)|\(Grønl.\)|\(Fær.\))", lang_taler[-2])[0])
                        tale.append(lang_taler[-1])
                else:
                    if len(tale) >= 1:
                        tale[-1] = tale[-1] + " " + sentence
            else: #ellers er det en tale
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
                #if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
        elif len(re.findall("(^.{1,50} \([A-Z]+\)|^.{1,50} \(KrF\)|^.{1,50} \(Grønl.\)|^.{1,50} \(Fær.\))", sentence)) > 0 and re.match("^Af ", sentence):
            talere.append(sentence)
            parti.append(re.findall("(\([A-Z]+\)|\(KrF\)|\(Grønl.\)|\(Fær.\))", sentence)[0])
            tale.append("")
        elif len(re.findall("(^.{1,50} \([A-Z]+\):|.{1,50} \(KrF\):)", sentence)) > 0:
            if len(sentence) > 35:
                if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
                    if len(re.findall("Ny stedf.", sentence)) > 0:
                        pass
                    elif len(re.findall("Udtræder:", sentence)) > 0:
                        pass
                    elif len(re.findall("Nyt medl", sentence)) > 0:
                        pass
                    elif len(re.findall(" af", sentence.lower())) > 0:
                        pass
                    elif re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0] in Partier_i_folketinget:
                        problem_menigt = re.split("(^.{1,50} \([A-Z]+\):|.{1,50} \(KrF\):)", sentence.lower())
                        talere.append(problem_menigt[-2])
                        parti.append("problem")
                        tale.append(problem_menigt[-1])
                    else:
                        if len(tale) >= 1:
                            tale[-1] = tale[-1] + " " + sentence
                else:
                    if len(tale) >= 1:
                        tale[-1] = tale[-1] + " " + sentence
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif sentence.lower() == "formanden" or sentence.lower() == "formanden:": #tjekker om det er formanden der taler
            talere.append("formanden")
            parti.append("Formanden")
            tale.append("")
        elif re.search("næstformand \(\w*\):$", sentence.lower()) or re.search("næstformand \(\w* \w*\):$", sentence.lower()):
            talere.append(sentence.lower())
            parti.append("næstformanden")
            tale.append("")
        elif len(sentence) < 100: #Tjekker om det er en minister der taler
            if len(re.findall("\w*minister[\w ]*\(([\w-]*|[\w ]*)\)", sentence.lower())) > 0:
                talere.append(sentence)
                parti.append("Regeringen")
                tale.append("")
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif len(sentence) >= 100:
            if len(re.findall("\w*minister[\w ]*\(([\w-]*|[\w ]*)\):", sentence.lower())) > 0:
                tale_med_minister = re.split("\w*minister[\w ]*\(([\w-]*|[\w ]*)\):", sentence.lower())
                talere.append(tale_med_minister[-2])
                parti.append("Regeringen")
                tale.append(tale_med_minister[-1])
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif len(re.findall("[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower()))  > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 170:
                pass
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif len(re.findall("^[1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: 
            if len(sentence) < 170:
                pass
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif re.fullmatch("\[.{1,100}\]", sentence.lower()):
            pass
        elif re.fullmatch("[1-9]{1,10}", sentence.lower()):
            pass
        else:
            if len(tale) >= 1:
                tale[-1] = tale[-1] + " " + sentence
    return tale, talere, parti

In [11]:
Folketingsår_opdelt_per_taler_80erne = {}
for år in Folketingstaler_i_80erne:
    indhold = Folketingstaler_i_80erne[år]
    år_tale, år_talere, år_parti = FT_taler(indhold)
    Folketingsår_opdelt_per_taler_80erne[år] = []
    Folketingsår_opdelt_per_taler_80erne[år].append(år_tale)
    Folketingsår_opdelt_per_taler_80erne[år].append(år_talere)
    Folketingsår_opdelt_per_taler_80erne[år].append(år_parti)

In [12]:
Folketingsår_opdelt_per_taler_90erne = {}
for år in Folketingstaler_i_90erne:
    indhold = Folketingstaler_i_90erne[år]
    år_tale, år_talere, år_parti = FT_taler(indhold)
    Folketingsår_opdelt_per_taler_90erne[år] = []
    Folketingsår_opdelt_per_taler_90erne[år].append(år_tale)
    Folketingsår_opdelt_per_taler_90erne[år].append(år_talere)
    Folketingsår_opdelt_per_taler_90erne[år].append(år_parti)

In [13]:
Folketingsår_opdelt_per_taler_00erne = {}
for år in Folketingstaler_i_00erne:
    indhold = Folketingstaler_i_00erne[år]
    år_tale, år_talere, år_parti = FT_taler(indhold)
    Folketingsår_opdelt_per_taler_00erne[år] = []
    Folketingsår_opdelt_per_taler_00erne[år].append(år_tale)
    Folketingsår_opdelt_per_taler_00erne[år].append(år_talere)
    Folketingsår_opdelt_per_taler_00erne[år].append(år_parti)

Der udvælges taler:

In [14]:
EU_ord_80_erne = [" eu ", " ef ", "europæiske fællesskab", "europæiske union", "fællesmarked", "ømu", " ef-", " eu-",
          "europæiske samarbejde", "fællesakt", " ømu", "det indre marked"]

In [15]:
eu_taler_80erne = {}
for år in Folketingsår_opdelt_per_taler_80erne:
    eu_tale_år = []
    eu_parti_år = []
    eu_taler_år = []
    for index in range(len(Folketingsår_opdelt_per_taler_80erne[år][0])):
        tale = Folketingsår_opdelt_per_taler_80erne[år][0][index]
        for ord_ in EU_ord_80_erne:
            if len(re.findall(ord_, tale.lower())) > 0:
                eu_tale_år.append(tale)
                eu_taler_år.append(Folketingsår_opdelt_per_taler_80erne[år][1][index])
                eu_parti_år.append(Folketingsår_opdelt_per_taler_80erne[år][2][index])
                break
    eu_taler_80erne[år] = []
    eu_taler_80erne[år].append(eu_tale_år)
    eu_taler_80erne[år].append(eu_parti_år)
    eu_taler_80erne[år].append(eu_taler_år)

In [16]:
EU_ord_90_erne = [" eu ", " ef ", "europæiske fællesskab", "europæiske union", "fællesmarked", "ømu", "maastricht", 
          "amsterdamtraktaten", "eu-", "ef-", "edinburgh", "indre marked"]

In [18]:
eu_taler_90erne = {}
for år in Folketingsår_opdelt_per_taler_90erne:
    eu_tale_år = []
    eu_parti_år = []
    eu_taler_år = []
    for index in range(len(Folketingsår_opdelt_per_taler_90erne[år][0])):
        tale = Folketingsår_opdelt_per_taler_90erne[år][0][index]
        for ord_ in EU_ord_90_erne:
            if len(re.findall(ord_, tale.lower())) > 0:
                eu_tale_år.append(tale)
                eu_parti_år.append(Folketingsår_opdelt_per_taler_90erne[år][1][index])
                eu_taler_år.append(Folketingsår_opdelt_per_taler_90erne[år][2][index])
                break
    eu_taler_90erne[år] = []
    eu_taler_90erne[år].append(eu_tale_år)
    eu_taler_90erne[år].append(eu_parti_år)
    eu_taler_90erne[år].append(eu_taler_år)

In [19]:
EU_ord_00_erne = [" eu ", " ef ", "europæiske fællesskab", "europæiske union", "fællesmarked", "ømu", "maastricht", 
          "amsterdamtraktaten", "eu-", "ef-", "edinburgh", "indre marked", "lissabon", "euro-", "euro "]

In [20]:
eu_taler_00erne = {}
for år in Folketingsår_opdelt_per_taler_00erne:
    eu_tale_år = []
    eu_parti_år = []
    eu_taler_år = []
    for index in range(len(Folketingsår_opdelt_per_taler_00erne[år][0])):
        tale = Folketingsår_opdelt_per_taler_00erne[år][0][index]
        for ord_ in EU_ord_00_erne:
            if len(re.findall(ord_, tale.lower())) > 0:
                eu_tale_år.append(tale)
                eu_parti_år.append(Folketingsår_opdelt_per_taler_00erne[år][1][index])
                eu_taler_år.append(Folketingsår_opdelt_per_taler_00erne[år][2][index])
                break
        if len(eu_tale_år) != 0:
            if eu_tale_år[-1] != tale:
                if len(re.findall("nice", tale.lower())) > 0:
                    if len(re.findall("unicef", tale.lower())) == 0:
                        eu_tale_år.append(tale)
                        eu_parti_år.append(Folketingsår_opdelt_per_taler_00erne[år][1][index])
                        eu_taler_år.append(Folketingsår_opdelt_per_taler_00erne[år][2][index])           
    eu_taler_00erne[år] = []
    eu_taler_00erne[år].append(eu_tale_år)
    eu_taler_00erne[år].append(eu_parti_år)
    eu_taler_00erne[år].append(eu_taler_år)

Det tjekkes om dette stemmer overens med antal udvalgte taler i de forrige scripts.

In [21]:
for år in eu_taler_80erne:
    print(f"{år}: {len(eu_taler_80erne[år][0])}")

1980: 507
1981: 594
1982: 510
1983: 526
1984: 423
1985: 868
1986: 608
1987: 673
1988: 942
1989: 998


In [22]:
for år in eu_taler_90erne:
    print(f"{år}: {len(eu_taler_90erne[år][0])}")

1990: 1093
1991: 1646
1992: 1729
1993: 1289
1994: 1699
1995: 2058
1996: 2374
1997: 2049
1998: 1868
1999: 2874


In [23]:
for år in eu_taler_00erne:
    print(f"{år}: {len(eu_taler_00erne[år][0])}")

2000: 2689
2001: 1940
2002: 2167
2003: 2434
2004: 2113
2005: 1321
2006: 1724
2007: 1775
2008: 1689


## Klargørring af data:

In [24]:
nlpd = spacy.load("da_core_news_lg")

In [25]:
tekster_klar_80 = []
tekster_klar_90 = []
tekster_klar_00 = []
fyldetegn = [",", ".", "(", ")", "?", "/", "!", ":", " ", "):", "-", "«", "»", "[", "]"]
stopord_fil = open('../../Folketingstidende/stopord.txt','r')
stopord = [word[:-1] for word in stopord_fil] + ["altså", "gå", "gerne", "tro", "faktisk", "gerne", "eu" "selvfølgelig", "dag"]

In [26]:
typer_spacy = []
for tegn in fyldetegn:
    tekst_nlpd = nlpd(tegn)
    for token in tekst_nlpd:
        if token.pos_ not in typer_spacy:
            typer_spacy.append(token.pos_)

In [27]:
typer_spacy

['PUNCT', 'X', 'SPACE', 'ADJ', 'PROPN']

In [28]:
typer_spacy.remove("PROPN")

In [29]:
for år in eu_taler_80erne:
    #allowed_postags=['NOUN', 'ADJ', 'VERB']
    start = time()
    for tekst in eu_taler_80erne[år][0]:
        tale_spacy = nlpd(tekst.lower())
        #tokens = [str(token.lemma_).lower() for token in tale_spacy]
        tokens = [token for token in tale_spacy]
        tokens_uden_tegn = [str(token) for token in tokens if token.pos_ not in typer_spacy]
        #tokens_uden_stopord = [token for token in tokens_uden_tegn if token not in stopord]
        tekster_klar_80.append(tokens_uden_tegn)
    end = time()
    print(end-start)

73.19719314575195
72.74021291732788
62.92735815048218
55.09942889213562
48.89830708503723
88.78709769248962
84.54854989051819
95.38612484931946
122.18778395652771
123.60365009307861


In [30]:
for år in eu_taler_90erne:
    #allowed_postags=['NOUN', 'ADJ', 'VERB']
    start = time()
    for tekst in eu_taler_90erne[år][0]:
        tale_spacy = nlpd(tekst.lower())
        #tokens = [str(token.lemma_).lower() for token in tale_spacy]
        tokens = [token for token in tale_spacy]
        tokens_uden_tegn = [str(token) for token in tokens if token.pos_ not in typer_spacy]
        #tokens_uden_stopord = [token for token in tokens_uden_tegn if token not in stopord]
        tekster_klar_90.append(tokens_uden_tegn)
    end = time()
    print(end-start)

147.5981092453003
170.45902490615845
186.8434157371521
162.2216911315918
172.53622198104858
195.55379629135132
249.21853375434875
193.13686990737915
175.84548497200012
247.98390293121338


In [31]:
for år in eu_taler_00erne:
    #allowed_postags=['NOUN', 'ADJ', 'VERB']
    start = time()
    for tekst in eu_taler_00erne[år][0]:
        tale_spacy = nlpd(tekst.lower())
        #tokens = [str(token.lemma_).lower() for token in tale_spacy]
        tokens = [token for token in tale_spacy]
        tokens_uden_tegn = [str(token) for token in tokens if token.pos_ not in typer_spacy]
        #tokens_uden_stopord = [token for token in tokens_uden_tegn if token not in stopord]
        tekster_klar_00.append(tokens_uden_tegn)
    end = time()
    print(end-start)

219.51706099510193
182.39954590797424
199.10082292556763
203.1259410381317
159.88194394111633
140.73420310020447
143.547669172287
155.23983597755432
162.54810523986816


In [32]:
processed_data_80 = []
for tekst in tekster_klar_80:
    str_tekst = [str(token) for token in tekst]
    processed_data_80.append(str_tekst)
processed_data_90 = []
for tekst in tekster_klar_90:
    str_tekst = [str(token) for token in tekst]
    processed_data_90.append(str_tekst)
processed_data_00 = []
for tekst in tekster_klar_00:
    str_tekst = [str(token) for token in tekst]
    processed_data_00.append(str_tekst)

In [33]:
bigram_80 = gensim.models.Phrases(processed_data_80, min_count=5, threshold=100)

trigram_80 = gensim.models.Phrases(bigram_80[processed_data_80], threshold=100)

bigram_mod_80 = gensim.models.phrases.Phraser(bigram_80)

trigram_mod_80 = gensim.models.phrases.Phraser(trigram_80)

In [34]:
bigram_90 = gensim.models.Phrases(processed_data_90, min_count=5, threshold=100)

trigram_90 = gensim.models.Phrases(bigram_90[processed_data_90], threshold=100)

bigram_mod_90 = gensim.models.phrases.Phraser(bigram_90)

trigram_mod_90 = gensim.models.phrases.Phraser(trigram_90)

In [35]:
bigram_00 = gensim.models.Phrases(processed_data_00, min_count=5, threshold=100)

trigram_00 = gensim.models.Phrases(bigram_00[processed_data_00], threshold=100)

bigram_mod_00 = gensim.models.phrases.Phraser(bigram_00)

trigram_mod_00 = gensim.models.phrases.Phraser(trigram_00)

In [36]:
def remove_stopwords(texts):
    return [[word for word in doc if word not in stopord] for doc in texts]
def create_bigrams_80(texts):
    return [bigram_mod_80[doc] for doc in texts]
def create_bigrams_90(texts):
    return [bigram_mod_90[doc] for doc in texts]
def create_bigrams_00(texts):
    return [bigram_mod_00[doc] for doc in texts]
def lemmatize(texts, allowed_postags=['NOUN', 'ADJ', 'VERB']):
    texts_op = []
    start = time()
    for sent in texts:
        doc = nlpd(" ".join(sent))
        texts_op.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    end = time()
    print(end-start)
    return texts_op

In [37]:
#removing stopwords, creating bigrams and lemmatizing the text

data_wo_stopwords_80 = remove_stopwords(processed_data_80)

data_bigrams_80 = create_bigrams_80(data_wo_stopwords_80)

data_lemmatized_80 = remove_stopwords(lemmatize(data_bigrams_80, allowed_postags=[ 'NOUN', 'ADJ', 'VERB', 'PROPN', 'ADV']))

291.34001088142395


In [38]:
#removing stopwords, creating bigrams and lemmatizing the text

data_wo_stopwords_90 = remove_stopwords(processed_data_90)

data_bigrams_90 = create_bigrams_80(data_wo_stopwords_90)

data_lemmatized_90 = remove_stopwords(lemmatize(data_bigrams_90, allowed_postags=[ 'NOUN', 'ADJ', 'VERB', 'PROPN', 'ADV']))

587.1515526771545


In [39]:
#removing stopwords, creating bigrams and lemmatizing the text

data_wo_stopwords_00 = remove_stopwords(processed_data_00)

data_bigrams_00 = create_bigrams_80(data_wo_stopwords_00)

data_lemmatized_00 = remove_stopwords(lemmatize(data_bigrams_00, allowed_postags=[ 'NOUN', 'ADJ', 'VERB', 'PROPN', 'ADV']))

492.2572979927063


In [40]:
texts_80 = [text for text in data_lemmatized_80]

texts_90 = [text for text in data_lemmatized_90]

texts_00 = [text for text in data_lemmatized_00]

shuffle(texts_80)

shuffle(texts_90)

shuffle(texts_00)

All_texts = texts_80 + texts_90 + texts_00

gensim_dictionary = corpora.Dictionary(All_texts)

gensim_corpus = [gensim_dictionary.doc2bow(text) for text in All_texts]

Gemmer dictionary

In [ ]:
gensim_dictionary.save("Dictionaries/Dynamic_model/05_01_23")

## Building model

In [ ]:
common_corpus = gensim_corpus[0:20] + gensim_corpus[len(texts_80):len(texts_80)+20] + gensim_corpus[len(texts_80) + len(texts_90):len(texts_80)+ len(texts_90)+20]

In [ ]:
start = time()
ldaseq = LdaSeqModel(corpus=common_corpus, time_slice=[20, 20, 20], num_topics=10, chunksize=1)
end = time()
print(end - start)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gensim/models/ldaseqmodel.py:297: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)


In [ ]:
ldaseq.save

## Building with BERTopic


In [41]:
data_p = data_lemmatized_80[0:10] + data_lemmatized_90[0:10] + data_lemmatized_00[0:10]

In [84]:
data_klar_p = []
for i in data_lemmatized_80:
    tale = ""
    for j in i:
        tale += j + " "
    data_klar_p.append(tale)
for i in data_lemmatized_90:
    tale = ""
    for j in i:
        tale += j + " "
    data_klar_p.append(tale)
for i in data_lemmatized_00:
    tale = ""
    for j in i:
        tale += j + " "
    data_klar_p.append(tale)

In [87]:
#from bertopic import BERTopic

start = time()
topic_model = BERTopic(language="multilingual", verbose=True)
topics, probs = topic_model.fit_transform(data_klar_p)
end = time()
print(end - start)

Batches:   0%|          | 0/1350 [00:00<?, ?it/s]

2023-01-07 23:25:16,941 - BERTopic - Transformed documents to Embeddings
2023-01-07 23:26:34,717 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable T

2023-01-07 23:26:41,484 - BERTopic - Clustered reduced embeddings


3251.8123922348022


In [86]:
len(data_klar_p)

43180

In [88]:
timestamps = [1980 for i in range(len(data_lemmatized_80))] + [1990 for i in range(len(data_lemmatized_90))] + [2000 for i in range(len(data_lemmatized_00))]

In [89]:
from bertopic import BERTopic
start = time()
topics_over_time = topic_model.topics_over_time(data_klar_p, timestamps, nr_bins=3)
end = time()

3it [00:23,  7.88s/it]


In [90]:
print(end - start)

24.284290075302124


In [91]:
topics_over_time

,Topic,Words,Frequency,Timestamp
0,-1,"ef, regering, forslag, hr, lov",3571,1979.980
1,0,"fiskeri, fisker, skib, fiskeriminister, fisker...",311,1979.980
2,1,"landbrug, landmand, landbrugsminister, landbru...",309,1979.980
3,2,"arbejdsmarked, arbejdsminister, arbejdstid, lø...",80,1979.980
4,3,"ordfører, hr, lovforslag, tak, forslag",91,1979.980
...,...,...,...,...
696,253,"transport, udsættersøer, transporter, transpor...",8,1993.333
697,254,"signalere, institutionerne, kompetenceændring,...",4,1993.333
698,255,"aktie, beskatning, domstolen, dom, selskab",4,1993.333
699,256,"elproducent, kraftværk, ordfører, miljøområde,...",6,1993.333


In [92]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,23101,-1_regering_danmark_hr_forslag
1,0,1317,0_fiskeri_fisker_skib_fisk
2,1,922,1_landbrug_landmand_landbrugsminister_landbrug...
3,2,715,2_arbejdsmarked_arbejdstager_arbejdsgiver_arbe...
4,3,653,3_ordfører_lovforslag_hr_tak
...,...,...,...
254,253,10,253_transport_udsættersøer_transporter_transpo...
255,254,10,254_maastrichttraktaten_tidsbegrænsning_garant...
256,255,10,255_dom_domstolen_landsret_tilbagebetaling
257,256,10,256_miljø_elproducent_ordfører_miljøområde
